In [1]:
import mxnet as mx
import numpy as np
import os
import time
import datetime

# Setting logger
import logging

ts = time.time()
timestamp = datetime.datetime.fromtimestamp(ts).strftime('%m-%d-%H-%M')

logfile = 'data/logs/player_tracking-{}.log'.format(timestamp)

logging.basicConfig(filename=logfile,level=logging.INFO)

def log_print(*args):
    print(" ".join(map(str, args)))
    logging.info(" ".join(map(str, args)))
    
log_print("logfile {} created and opened".format(logfile))


logfile data/logs/warper-07-11-18-37.log created and opened


### Configurable Params

In [ ]:
#warper model params
path_to_warper_weights = 'data/mx_ckpts/warper-07-11-18-37-epoch'
weights_epoch = 800
target_shape = (3, 300, 300)
input_data_name = 'data'
output_data_name = 'reg_out'

mean_color = np.array([0.6274094, 0.61885411, 0.61518264], dtype=float)






### Initializing the warper module

In [34]:
# Get the module
from vistools.inceptionv3 import get_symbol

warp_sym = get_symbol(num_outputs=9, output_name=output_data_name)

warp_mod = mx.mod.Module(symbol=sym,
                        context=context,
                        data_names=[input_data_name],
                        label_names=[output_data_name])
log_print("Warper Module initialized")

Module initialized


In [35]:
# Load weights
log_print('Using {} weights'.format(path_to_weights))
log_print('CONTINUED TRAINING from epoch {}'.format(weights_epoch))
_, arg_params, aux_params = mx.model.load_checkpoint(path_to_weights, weights_epoch)
warp_mod.bind(for_training=False, data_shapes=img_iter.provide_data, 
         label_shapes=img_iter.provide_label)
warp_mod.set_params(arg_params, aux_params, allow_missing=False, allow_extra=False, force_init=True)

log_print("Params loaded")

Using data/mx_ckpts/warper-07-11-18-37-epoch weights
CONTINUED TRAINING from epoch 800
Params loaded


In [36]:
# Define callbacks
epochs_per_ckpt = 100
prefix = 'data/mx_ckpts/warper-{}-epoch'.format(timestamp)

checkpointer = mx.callback.do_checkpoint(prefix, epochs_per_ckpt)
batches_per_epoch = len(train_imglist) / batch_size
progress_bar = mx.callback.ProgressBar(total=batches_per_epoch)
log_print('Defined callbacks')

Defined callbacks


### Initializing detection module

### Running predictions

In [ ]:
# Predict using each module and combine results for player tracking
from utils.warp_tools import src_xy_to_dst_xy

img_iter.reset()
for i, batch in enumerate(img_iter):
    player_mod.forward(batch, is_train=False)    
    warp_mod.forward(batch, is_train=False)       # compute predictions
    warp_batch_pred = warp_mod.get_outputs()[0]
    player_batch_pred = warp_mod.get_outputs()[0]

    for j, warp_pred, player_pred in enumerate(zip(warp_batch_pred, player_batch_pred)):
        H_unshaped = np.array(warp_pred.asnumpy(), dtype=np.float32)
        H = H_unshaped.reshape((3,3))
        
        player_locs = player_pred
        
        dst_xy = src_xy_to_dst_xy()
        